### Installation

In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab and Kaggle notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
    !pip install --no-deps cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [29]:
from unsloth import FastLanguageModel, to_sharegpt
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [30]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [31]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("data", split = "train")


dataset = to_sharegpt(
    dataset,
    merged_prompt = "{instruction}[[\nYour input is:\n{input}]]",
    output_column_name = "output",
    conversation_extension = 3, # 3 筆資料變成一個對話的形式
)


Merging columns:   0%|          | 0/62 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/62 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/62 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/62 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/62 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/62 [00:00<?, ? examples/s]

In [32]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Standardizing format:   0%|          | 0/62 [00:00<?, ? examples/s]

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

We look at how the conversations are structured for item 5:

In [33]:
dataset[5]["conversations"]

[{'content': '2～3萬多PS修圖用（內置光碟機）', 'role': 'user'},
 {'content': 'CPU (中央處理器)：intel i5 12400（6核/12緒）2.5G\nMB      (主機板)：微星 B760M BOMBER DDR4\nRAM     (記憶體)：威剛16GDDR4-3200, 之前修圖到一定程度會當機卡住，是否記憶體要\n多買點？\nVGA     (顯示卡)：技嘉1650ti（2023年才買的，續用）\nCooler  (散熱器)：\nSSD   (固態硬碟)：原本107年買的SSD剛過保考慮想換新，降低故障風險\nHDD       (硬碟)：有SSD還需要HDD嗎？\nPSU (電源供應器)：全漢聖武士550W銅牌\nCHASSIS   (機殼)：偏好白色，內置光碟機必備，不要太舊款，畢竟要再戰10年，要有ty\npe-c\nMONITOR   (螢幕)：EIZO EV2456（續用）\nOS    (作業系統)：原有win7升到10，會買正版win11\n\n光碟機      (自填)：ASUS續用\n總價 (未稅/含稅)：',
  'role': 'assistant'},
 {'content': '80K 遊戲主機', 'role': 'user'},
 {'content': 'CPU (中央處理器)：Intel Core Ultra 7 265K\nMB      (主機板)：華碩 TUF GAMING Z890-PLUS\nRAM     (記憶體)：威剛 64GB(32GB*2) DDR5 5600 XPG Lancer Blade(黑)\nVGA     (顯示卡)：下個月加上40K的5080顯卡\nCooler  (散熱器)：喬思伯 PISA A5 灰\nSSD   (固態硬碟)：威剛 XPG S60 PRO 2TB/Gen4/讀:5000M/寫:4500M\nHDD       (硬碟)：現有沿用\nPSU (電源供應器)：LEADEX VII PRO 850W/白金\nCHASSIS   (機殼)：華碩 TUF Gaming GT302 ARGB 黑\nMONITOR   (螢幕)：現有沿用\nMouse/KB  (鼠鍵)：現有沿用\nOS    (作業系統)：\n\n其它

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [35]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/62 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/62 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/62 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/62 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs.

In [36]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/62 [00:00<?, ? examples/s]

In [39]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
4.805 GB of memory reserved.


In [40]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 62 | Num Epochs = 9
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,2.746500
2,2.719900
3,2.680500
4,2.652400
5,2.580100
6,2.408500
7,2.309900
8,4.642200
9,2.265900
10,2.198700


In [41]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

731.5933 seconds used for training.
12.19 minutes used for training.
Peak reserved memory = 4.805 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 32.596 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

**[NEW] Try 2x faster inference in a free Colab for Llama-3.1 8b Instruct [here](https://colab.research.google.com/drive/1T-YBVfnphoVc8E2E854qF3jdia2Ll2W2?usp=sharing)**

We use `min_p = 0.1` and `temperature = 1.5`. Read this [Tweet](https://x.com/menhguin/status/1826132708508213629) for more information on why.

In [46]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "我想要一台三萬元的遊戲機"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 128, use_cache = True, temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)
# print(tokenizer.batch_decode(outputs)[0])

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n我想要一台三萬元的遊戲機<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nCPU (中央處理器)：【沿用】AMD【8核】Ryzen7 7700\nMB      (主機板)：【降價】華碩 TUF GAMING B650M-E\nRAM     (記憶體)：【降價】十銓 Team Deathmatch DD5-4000\nVGA     (顯示卡)：\nCooler  (散熱器)：利民利民 T500\nSSD   (固態硬碟)：\nHDD       (硬碟)：\nPSU (電源供應器)：']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [49]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "我想要一台三萬元的遊戲機"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 256,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

CPU (中央處理器)：【獨家推品 】 AMD R7 9800X 8核/16緒/3.8GHz(彌射)【6.5G】/80plus
金牌/雙風扇/內含風扇散熱器/代理商年保，$12,990

MB      (主機板)：Aorus X870G E-ATX,M-ATX主機板 2100X/Realtek 2.5G/WiFi 6.2相電腦
風扇上蓋(註冊價)，$6990

RAM     (記憶體)：Aorus XENON DDR5-6400/CL32-128GX EDC(註冊價)、
$10997

VGA     (顯示卡)：MSI RX 6650 XT TRIO OC 8GB+AMD 9700 X70(【規搭）, $14,690

SSD   (固態硬碟)：美光 T500 1TB/Gen5/讀:7000M/寫:7000M(含機殼外盒)
